In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow, Flow
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError
import json
from google.oauth2.credentials import Credentials
import os
from pathlib import Path
from elasticsearch import Elasticsearch
import json
import pandas as pd

In [2]:
class YouTubeAPIOauthConstants:
    TOKEN_FILE = 'credentials.json'
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'
    SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [3]:
class YouTubeAPIAuth:
    __TOKEN_FILE = YouTubeAPIOauthConstants.TOKEN_FILE
    __API_SERVICE_NAME = YouTubeAPIOauthConstants.API_SERVICE_NAME
    __API_VERSION = YouTubeAPIOauthConstants.API_VERSION
    __SCOPES = YouTubeAPIOauthConstants.SCOPES
    
    def __init__(self):
        self.__credentials_path = None
        self.__client_secrets_file = None
        self.__credentials = None
    
    def authenticate_from_client_secrets_file(self, client_secrets_file: str, 
                                              credentials_path: str = ''):
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        if not credentials_path or not os.path.exists(credentials_path):
            self.__credentials_path = self.__get_default_credentials_path()
        else:
            self.__credentials_path = credentials_path
        return self.__from_client_secrets_file()
    
    def authenticate_from_credentials(self, credentials_path: str):
        if not credentials_path:
            raise ValueError('The credentials file path has to be provided.')
        if not isinstance(credentials_path, str):
            raise TypeError('The credentials file should be a string.')
        if not os.path.exists(credentials_path):
            raise ValueError('The credentials file path has to exist!')
        if not Path(credentials_path).is_file():
            raise ValueError('The credentials path must be a file.')
        with open(credentials_path, "r") as credentials:
            self.__credentials = Credentials(**json.load(credentials))
        return self.__from_credentials()
            
    
    def __verify_client_secret_file(self, client_secrets_file: str) -> None:
        """Verfy the client secret file."""
        if not client_secrets_file:
            raise ValueError('The clients secret file path has to be provided.')
        if not isinstance(client_secrets_file, str):
            raise TypeError('The clients secret file should be a string.')
        if not os.path.exists(client_secrets_file):
            raise ValueError(f'The path {client_secrets_file} does not exist!')
            
    def __get_default_credentials_path(self):
        """Generate the default api token file location."""
        current_user_home_dir = os.path.expanduser('~')
        credentials_path = os.path.join(current_user_home_dir, self.__TOKEN_FILE)
        return credentials_path
    
    def __from_client_secrets_file(self):
        if os.path.exists(self.__credentials_path):
            with open(self.__credentials_path, "r") as credentials:
                self.__credentials = Credentials(**json.load(credentials))
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.__client_secrets_file, self.__SCOPES)
                self.__credentials = flow.run_local_server(port=0)
            with open(self.__credentials_path, "w") as credentials_path:
                credentials = self.__credentials_to_dict(self.__credentials)
                json.dump(credentials, credentials_path)
        youtube_api_client = build(self.__API_SERVICE_NAME, self.__API_VERSION, 
                                   credentials=self.__credentials)
        return youtube_api_client
    
    def __from_credentials(self):
        if not self.__credentials or not self.__credentials.valid:
            if self.__credentials and self.__credentials.expired and self.__credentials.refresh_token:
                self.__credentials.refresh(Request())
            with open(self.__credentials_path, "w") as credentials_path:
                credentials = self.__credentials_to_dict(self.__credentials)
                json.dump(credentials, credentials_path)
        youtube_api_client = build(self.__API_SERVICE_NAME, self.__API_VERSION, 
                                   credentials=self.__credentials)
        return youtube_api_client
    
    def generate_credentials(self, client_secrets_file: str, credentials_path: str = ''):
        self.__verify_client_secret_file(client_secrets_file)
        self.__client_secrets_file = client_secrets_file
        if not credentials_path or not os.path.exists(credentials_path):
            self.__credentials_path = self.__get_default_credentials_path()
        else:
            self.__credentials_path = credentials_path
        flow = Flow.from_client_secrets_file(
            self.__client_secrets_file,
            scopes=self.__SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob')
        auth_url, _ = flow.authorization_url(prompt='consent')

        print('Please go to this URL: {}'.format(auth_url))
        code = input('Enter the authorization code: ')
        flow.fetch_token(code=code)
        self.__credentials = flow.credentials
        credentials_dict = self.__credentials_to_dict(self.__credentials)
        with open(self.__credentials_path, "w") as credentials_path:
            json.dump(credentials_dict, credentials_path)
    
    def __credentials_to_dict(self, credentials: Credentials) -> dict:
        """Convert credentials to a dict for easy work with Flask."""
        return dict(
            token=credentials.token,
            refresh_token=credentials.refresh_token,
            token_uri=credentials.token_uri,
            client_id=credentials.client_id,
            client_secret=credentials.client_secret,
            scopes=credentials.scopes
        )

In [4]:
class YouTubeSearchQuery:
    """A query to pass to the search resource."""
    
    def __init__(self, query_string: str):
        self.__query_string = query_string
        
    @property
    def query_string(self):
        return self.__query_string
    
    @query_string.setter
    def query_string(self, query_str: str):
        if not query_str:
            raise ValueError('The query string has to be provided')
        if not isinstance(query_str, str):
            raise TypeError('The query string has to be a string')
        self.__query_string = query_str

In [5]:
class YouTubeVideoSearchQuery(YouTubeSearchQuery):
    pass

In [6]:
class YouTubeSearchType:
    VIDEO = 'video'
    CHANNEL = 'channel'
    PLAYLIST = 'playlist'

In [7]:
class YouTubeSearch:
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, *args, **kwargs):
        pass

In [18]:
class VideoStat:
    def __init__(self, stat: int):
        self.__stat = stat
     
   #stat setter 
    def stat(self):
        pass
    
    #stat getter
    def get_stat():
        pass
    
    def __format_stat(self):
        pass

In [8]:
class YouTubeVideoStats:
    def __init__(self, viewCount: int, likeCount: int, commentCount: int):
        self.__view_count = int(viewCount)
        self.__like_count = int(likeCount)
        self.__comment_count = int(commentCount)
        
    def get_video_stats(self):
        video_stats = {
            'view_count': self.__view_count,
            'like_count': self.__like_count,
            'comment_count': self.__comment_count
        }
        return video_stats

In [40]:
class YouTubeVideoDetails:
    def __init__(self, id: str, channelId: str, title: str, channelTitle: str, 
                 description: str, thumbnails: str, tags: list[str], duration: str, licensedContent: bool):
        self.__id = id
        self.__channel_id = channelId
        self.__title = title
        self.__channel_title = channelTitle
        self.__description = description
        self.__thumbnails = thumbnails
        self.__tags = tags
        self.__duration = duration
        self.__licensed_content = licensedContent
        
    def get_video_details(self):
        video_details = {
            'id': self.__id,
            'channel_id': self.__channel_id,
            'title': self.__title,
            'channel_title': self.__channel_title,
            'description': self.__description,
            'thumbnail': self.get_video_thumbnail(),
            'tags': self.__tags,
            'duration': self.__duration,
            'licensed_content': self.__licensed_content
        }
        return video_details
    
    def get_video_thumbnail(self):
        thumbnail = ''
        if self.__thumbnails:
            if self.__thumbnails.get('default'):
                thumbnail = self.__thumbnails.get('default').get('url')
            elif self.__thumbnails.get('medium'):
                thumbnail = self.__thumbnails.get('medium').get('url')
            elif self.__thumbnails.get('high'):
                thumbnail = self.__thumbnails.get('high').get('url')
            elif self.__thumbnails.get('standard'):
                thumbnail = self.__thumbnails.get('standard').get('url')
            elif self.__thumbnails.get('maxres'):
                thumbnail = self.__thumbnails.get('maxres').get('url')
        return thumbnail
    
    def get_video_title(self):
        return self.__title
    
    def get_video_id(self):
        return self.__id
    
    def get_video_description(self):
        return self.__description
    
    def get_video_duration(self):
        pass
    
    def get_video_tags(self):
        if self.__tags:
            return self.__tags
        return []
    
    def get_channel_id(self):
        return self.__channel_id
    
    def get_channel_title(self):
        return self.__channel_title

In [41]:
class YouTubeComment:
    def __init__(self, id: str, videoId: str, totalReplyCount: str, textDisplay: str, 
                authorDisplayName: str, authorProfileImageUrl: str, authorChannelId: str,
                likeCount: str, publishedAt: str, updatedAt: str):
        self.__id = id
        self.__video_id = videoId
        self.__total_reply_count = totalReplyCount
        self.__text_display = textDisplay
        self.__author_display_name = authorDisplayName
        self.__author_profile_image_url = authorProfileImageUrl
        self.__author_channel_id = authorChannelId
        self.__like_count = int(likeCount)
        self.__published_at = publishedAt
        self.__updated_at = updatedAt
        
    def get_comment(self):
        comment = dict(
            id=self.__id,
            videoId=self.__video_id,
            totalReplyCount=self.__total_reply_count,
            textDisplay=self.__text_display,
            authorDisplayName = self.__author_display_name,
            authorProfileImageUrl = self.__author_profile_image_url,
            authorChannelId = self.__author_channel_id,
            likeCount = self.__like_count,
            publishedAt = self.__published_at,
            updatedAt = self.__updated_at
        )
        return comment
    
    def get_comment_text(self):
        comment_text = self.__text_display
        return comment_text
    
    def __str__(self):
        return self.get_comment_text()
    
    def __repr__(self):
        return f"YouTubeComment(id='{self.__id}', videoId='{self.__video_id}', \
        totalReplyCount={self.__total_reply_count})"

In [42]:
class YouTubeCommentThread:
    def __init__(self, video_id: str):
        self.__video_id = video_id
        
    def get_video_comments(self, youtube_client):
        """Get the top level comments for a video."""
        youtube_comments = self.__find_comments(youtube_client)
        youtube_comments = [self.__create_comment(comment) for comment in youtube_comments]
        return youtube_comments
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            videoId=self.__video_id,
            part='snippet,replies'
        ) 
        return basic_info_params
    
    def __find_comments(self, youtube_client):
        """Find the video comments."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.commentThreads().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        comments = self.__parse_comments(search_response)
        return comments
    
    def __create_comment(self, comment_details):
        youtube_comment = YouTubeComment(**comment_details)
        return youtube_comment

    def __parse_comments(self, search_response):
        items = search_response['items']
        comments = []
        for item in items:
            comments.append({
                'id': item['id'],
                'videoId': item['snippet']['videoId'],
                'totalReplyCount': item['snippet']['totalReplyCount'],
                'textDisplay': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                'authorDisplayName': item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                'authorProfileImageUrl': item['snippet']['topLevelComment']['snippet']['authorProfileImageUrl'],
                'authorChannelId': item['snippet']['topLevelComment']['snippet']['authorChannelId']['value'],
                'likeCount': item['snippet']['topLevelComment']['snippet']['likeCount'],
                'publishedAt': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                'updatedAt': item['snippet']['topLevelComment']['snippet']['updatedAt']
        })
        return comments

In [98]:
class YouTubeVideo:
    """A YouTube Video."""
    def __init__(self, video_details, youtube_client):
        self.__youtube_client = youtube_client
        self.__video_stats = self.__create_video_stats(video_details)
        self.__video_details = self.__create_video_details(video_details)
        self.__video_top_level_comments = self.__set_video_comments()
        self.__channel = self.__set_video_channel()
        
    def set_youtube_client(self, youtube_client):
        self.__youtube_client = youtube_client
        
    def get_video_stats_details(self):
        video_stats_details = dict()
        video_stats_details['details'] = self.get_video_details()
        video_stats_details['statistics'] = self.get_video_stats()
        return video_stats_details
    
    def __set_video_comments(self):
        youtube_commenthread = YouTubeCommentThread(self.get_video_id())
        video_top_level_comments = youtube_commenthread.get_video_comments(self.__youtube_client)
        return video_top_level_comments
    
    def __set_video_channel(self):
        find_channel = FindChannel(self.__youtube_client)
        video_channel = find_channel.find_channel(self.get_channel_id())
        return video_channel
    
    def get_video_comments(self):
        if not self.__video_top_level_comments:
            self.__set_video_comments()
        return self.__video_top_level_comments
        
    def __create_video_stats(self, video_details: dict):
        video_stats = YouTubeVideoStats(**video_details['statistics'])
        return video_stats
    
    def __create_video_details(self, video_details: dict):
        video_details = YouTubeVideoDetails(**video_details['details'])
        return video_details
        
    def get_video_stats(self):
        return self.__video_stats.get_video_stats()
    
    def get_video_details(self):
        return self.__video_details.get_video_details()
    
    def get_video_id(self):
        return self.__video_details.get_video_id()
    
    def get_video_title(self):
        return self.__video_details.get_video_title()
    
    def get_video_description(self):
        return self.__video_details.get_video_description()
    
    def get_video_tags(self):
        return self.__video_details.get_video_tags()
    
    def get_channel_id(self):
        return self.__video_details.get_channel_id()
    
    def get_channel_title(self):
        return self.__video_details.get_channel_title()
    
    def get_video_thumbnail(self):
        return self.__video_details.get_video_thumbnail()
    
    def get_video_channel_thumbnail(self):
        if not self.__channel:
            self.__set_video_channel()
        return self.__channel.get_channel_thumbnail()
    
    def to_dict(self):
        return {
            'video_id': self.get_video_id(),
            'video_title': self.get_video_id(),
            'video_description': self.get_video_description(),
            'video_thumbnail': self.get_video_thumbnail(),
            'video_tags': self.get_video_tags(),
            'channel_id': self.get_channel_id(),
            'channel_title': self.get_channel_title(),
            'channel_thumbnail': self.get_video_channel_thumbnail()
        }
    
    def to_csv(self):
        pass
    
    def __str__(self):
        return f'{self.get_video_title()} from {self.get_channel_title()}'
    
    def __repr__(self):
        return "YouTubeVideo(video_details={})"

In [80]:
class YouTubeVideoCollection:
    def __init__(self):
        self.__youtube_videos = []
        self.__es_client = None
        
    def get_youtube_videos(self):
        return self.__youtube_videos
    
    def add_video(self, video: YouTubeVideo):
        self.__youtube_videos.append(video)
        
    def add_videos(self, videos: list[YouTubeVideo]):
        for video in videos:
            self.__youtube_videos.append(video)
    
    def save_to_database(self):
        pass
    
    def save_to_elasticsearch(self, index_name, elastic_search_host):
        if not self.__es_client:
            self.__create_es_client(elastic_search_host)
        for video in self.__youtube_videos:
            video_details = video.to_dict()
            self.__es_client.index(index=index_name, document=video_details, id=video_details['video_id'])
        
    def __delete_index(self, index_name):
        self.__es_client.indices.delete(index=index_name, ignore=[400, 404])
        
    def __create_index(self, index_name):
        self.__es_client.indices.create(index = index_name)
    
    def __create_es_client(self, es_host):
        if not self.__es_client:
            self.__es_client = Elasticsearch(hosts=[es_host])
    
    def save_to_csv(self):
        pass
    
    def save_to_json(self, file=''):
        if not file:
            file = 'file.json'
        with open(file, 'w') as file_path:
            videos = [video.to_dict() for video in self.__youtube_videos]
            json.dump(videos)
    
    def to_pandas(self):
        videos = [video.to_dict() for video in self.__youtube_videos]
        video_ids = []
        video_titles = []
        video_descriptions = []
        video_thumbnails = []
        video_tags = []
        channel_ids = []
        channel_titles = []
        for video in videos:
            video_ids.append(video['video_id'])
            video_titles.append(video['video_title'])
            video_descriptions.append(video['video_description'])
            video_thumbnails.append(video['video_thumbnail'])
            video_tags.append(video['video_tags'])
            channel_ids.append(video['channel_id'])
            channel_titles.append(video['channel_title'])
        data = {
            'video_id': video_ids,
            'video_title': video_titles,
            'video_description': video_descriptions,
            'video_thumbnail': video_thumbnails,
            'video_tags': video_tags,
            'channel_id': channel_ids,
            'channel_title': channel_titles
        }

        df = pd.DataFrame(data)

In [92]:
class FindVideo:
    def __init__(self, youtube_client):
        """Find the video with the given id."""
        self.__youtube_client = youtube_client
        
    def __generate_basic_info_params(self, video_id: str):
        basic_info_params = dict(
            id=video_id,
            part='snippet,contentDetails,statistics'
        ) 
        return basic_info_params
    
    def find_video(self, video_id: str):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params(video_id)
        search_request = self.__youtube_client.videos().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_video_details(search_response)
        youtube_video = YouTubeVideo(parsed_response, self.__youtube_client)
        return youtube_video
    
    def __parse_video_details(self, video_details: dict):
        """Parse the video details.

        Returns
        -------
        parsed_video_details: dict
            A dictionary of the YouTube video details.
        """
        parsed_video_details = dict()
        items = video_details['items'][0]
        parsed_video_details['details'] = dict()
        parsed_video_details['statistics'] = dict()
        parsed_video_details['details']['id'] = items['id']
        parsed_video_details['details']['channelId'] = items['snippet']['channelId']
        parsed_video_details['details']['title'] = items['snippet']['title']
        parsed_video_details['details']['channelTitle'] = items['snippet']['channelTitle']
        parsed_video_details['details']['description'] = items['snippet']['description']
        parsed_video_details['details']['thumbnails'] = items['snippet']['thumbnails']
        if items['snippet'].get('tags'):
            parsed_video_details['details']['tags'] = items['snippet']['tags']
        else:
            parsed_video_details['details']['tags'] = []
        parsed_video_details['details']['duration'] = items['contentDetails']['duration']
        parsed_video_details['details']['licensedContent'] = items['contentDetails']['licensedContent']
        parsed_video_details['statistics']['viewCount'] = items['statistics']['viewCount']
        parsed_video_details['statistics']['likeCount'] = items['statistics']['likeCount']
        parsed_video_details['statistics']['commentCount'] = items['statistics']['commentCount']
        return parsed_video_details

In [71]:
class VideoSearch(YouTubeSearch):
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, youtube_client):
        self.__youtube_client = youtube_client
        self.__type = YouTubeSearchType.VIDEO
        self.__query = ''
        self.__videos = YouTubeVideoCollection()
        
    def __get_query(self):
        return self.__query.query_string
    
    def get_videos(self):
        return self.__videos
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.__MAX_RESULTS,
            regionCode=self.__REGION_CODE
        ) 
        return basic_info_params
    
    def __generate_basic_info_params_for_related_video(self, video_id):
        basic_info_params = dict(
            part='id,snippet',
            type=self.__type,
            relatedToVideoId=video_id,
            maxResults=self.__MAX_RESULTS,
            regionCode=self.__REGION_CODE
        ) 
        return basic_info_params
    
    def __generate_basic_info_params_for_most_popular_video_by_region(self, 
                region_code):
        basic_info_params = dict(
            part='id,snippet',
            chart='mostPopular',
            regionCode=region_code
        ) 
        return basic_info_params
    
    def __generate_basic_info_params_for_most_popular_video_by_category(self, 
                category_id):
        basic_info_params = dict(
            part='id,snippet',
            chart='mostPopular',
            videoCategoryId=category_id
        ) 
        return basic_info_params
    
    def search_videos(self, query_string: str, next_page_token=None):
        self.__query = YouTubeSearchQuery(query_string)
        basic_info_params = self.__generate_basic_info_params()
        if next_page_token:
            basic_info_params['pageToken'] = next_page_token
        search_request = self.__youtube_client.search().list(
            **basic_info_params
        )
        search_response = search_request.execute()
        (previous_page_token, next_page_token, videos) = self.__parse_basic_response(search_response)
        return (previous_page_token, next_page_token, videos)
    
    def __parse_basic_response(self, search_response):
        videos = []
        previous_page_token = search_response.get('prevPageToken', '')
        next_page_token = search_response.get('nextPageToken', '')
        video_results = search_response['items']
        for video_result in video_results:
            video_id = video_result['id']['videoId']
            youtube_video = FindVideo(self.__youtube_client).find_video(video_id)
            videos.append(youtube_video)
        self.__videos.add_videos(videos)
        return (previous_page_token, next_page_token, videos)
    
    def __parse_basic_response_popular_videos(self, search_response):
        videos = []
        video_results = search_response['items']
        for video_result in video_results:
            if video_result.get('id'):
                video_id = video_result['id']
            else:
                video_id = video_result['id']['videoId']
            youtube_video = FindVideo(video_id).find_video(self.__youtube_client)
            videos.append(youtube_video)
        return videos
    
    def search_related_videos(self, youtube_video, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params_for_related_video(youtube_video.get_video_id())
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            videos = self.__parse_basic_response(search_response, youtube_client)
        return videos
    
    def search_most_popular_videos_by_region(self, region_code, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params_for_most_popular_video_by_region(region_code)
            search_request = youtube_client.videos().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            videos = self.__parse_basic_response_popular_videos(search_response, youtube_client)
        return videos
    
    def search_most_popular_videos_by_category(self, category_id, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params_for_most_popular_video_by_category(category_id)
            search_request = youtube_client.videos().list(
                **basic_info_params
            )
            try:
                search_response = search_request.execute()
                videos = self.__parse_basic_response_popular_videos(search_response, youtube_client)
            except:
                videos = []
        return videos

In [47]:
class YouTubeVideoCategory:
    def __init__(self, id, title):
        self.__id = id
        self.__title = title
        
    def get_id(self):
        return self.__id
    
    def get_title(self):
        return self.__title
    
    def __str__(self):
        return self.get_title()
    
    def __repr__(self):
        return f"YouTubeVideoCategory(id='{self.__id}', title='{self.__title}')"

In [48]:
class SearchYouTubeVideoCategories:
    def __init__(self, youtube_client, region_code='us'):
        self.__youtube_client = youtube_client
        self.__region_code = region_code
        self.__youtube_video_categories = self.__search_video_categories()
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='snippet',
            regionCode=self.__region_code
        )
        return basic_info_params
    
    def __search_video_categories(self):
        basic_info_params = self.__generate_basic_info_params()
        search_request = self.__youtube_client.videoCategories().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        video_categories = self.__parse_categories(search_response)
        return video_categories
    
    def __parse_categories(self, search_response):
        category_data = []
        items = search_response['items']
        for item in items:
            category = dict()
            category['id'] = item['id']
            category['title'] = item['snippet']['title']
            category_data.append(YouTubeVideoCategory(**category))
        return category_data
    
    def get_youtube_video_categories(self):
        return self.__youtube_video_categories

In [14]:
class PlaylistItem:
    def __init__(self, id, publishedAt, channelId, title, description, thumbnails, 
                channelTitle, position, videoId, videoOwnerChannelTitle, 
                videoOwnerChannelId):
        self.__id = id
        self.__publishedAt = publishedAt
        self.__channelId = channelId
        self.__title = title
        self.__description = description
        self.__thumbnails = thumbnails
        self.__channelTitle = channelTitle
        self.__position = int(position)
        self.__videoId = videoId
        self.__videoOwnerChannelTitle = videoOwnerChannelTitle
        self.__videoOwnerChannelId = videoOwnerChannelId
        self.__video = None
        
    def get_video(self, youtube_client):
        if not self.__video:
            self.__video = FindVideo(self.__videoId).find_video(youtube_client)
        return self.__video

In [15]:
class PlayList:
    def __init__(self, id, channelId, title, description, thumbnails, channelTitle, 
                itemCount, player):
        self.__id = id
        self.__channelId = channelId
        self.__title = title
        self.__description = description
        self.__thumbnails = thumbnails
        self.__channelTitle = channelTitle
        self.__itemCount = itemCount
        self.__player = player
        self.__playlist_items = []
        self.__videos = []
        
    def get_playlist_items(self, youtube_client):
        if not self.__playlist_items:
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.playlistItems().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            parsed_response = self.__parse_playlist_items(search_response)
            self.__playlist_items = [PlaylistItem(**item) for item in parsed_response]
        return self.__playlist_items
    
    def get_videos(self, youtube_client):
        if not self.__videos:
            play_list_items = self.get_playlist_items(youtube_client)
            for playlist_item in play_list_items:
                self.__videos.append(playlist_item.get_video(youtube_client))
        return self.__videos
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id,snippet,contentDetails',
            playlistId=self.__id
        ) 
        return basic_info_params
    
    def __parse_playlist_items(self, search_response):
        playlist_items = []
        items = search_response['items']
        for item in items:
            playlist_item = dict()
            playlist_item['id'] = item['id']
            playlist_item['publishedAt'] = item['snippet']['publishedAt']
            playlist_item['channelId'] = item['snippet']['channelId']
            playlist_item['title'] = item['snippet']['title']
            playlist_item['description'] = item['snippet']['description']
            playlist_item['thumbnails'] = item['snippet']['thumbnails']
            playlist_item['channelTitle'] = item['snippet']['channelTitle']
            playlist_item['position'] = item['snippet']['position']
            playlist_item['videoId'] = item['snippet']['resourceId']['videoId']
            playlist_item['videoOwnerChannelTitle'] = item['snippet']['videoOwnerChannelTitle']
            playlist_item['videoOwnerChannelId'] = item['snippet']['videoOwnerChannelId']
            playlist_items.append(playlist_item)
        return playlist_items
    
    def get_playlist_thumbnail(self):
        return self.__thumbnails['standard']['url']
    
    def get_playlist_title(self):
        return self.__title
    
    def get_playlist_channel_title(self):
        pass

In [16]:
class FindPlaylist:
    def __init__(self, playlist_id: str):
        """Find the video with the given id."""
        self.__playlist_id = playlist_id
        
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            id=self.__playlist_id,
            part='id,snippet,contentDetails,player',
        ) 
        return basic_info_params
    
    def find_playlist(self, youtube_client):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params()
        search_request = youtube_client.playlists().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_playlist(search_response)
        youtube_playlist = PlayList(**parsed_response)
        return youtube_playlist
    
    def __parse_playlist(self, search_response):
        playlist_details = dict()
        items = search_response['items'][0]
        playlist_details['id'] = items['id']
        playlist_details['channelId'] = items['snippet']['channelId']
        playlist_details['title'] = items['snippet']['title']
        playlist_details['description'] = items['snippet']['description']
        playlist_details['thumbnails'] = items['snippet']['thumbnails']
        playlist_details['channelTitle'] = items['snippet']['channelTitle']
        playlist_details['itemCount'] = items['contentDetails']['itemCount']
        playlist_details['player'] = items['player']['embedHtml']
        return playlist_details

In [17]:
class PlaylistSearch(YouTubeSearch):
    __MAX_RESULTS = 10
    __REGION_CODE = 'US'
    
    def __init__(self, query_string: str):
        self.__type = YouTubeSearchType.PLAYLIST
        self.__query = YouTubeSearchQuery(query_string)
        
    def __get_query(self):
        return self.__query.query_string
        
    def basic_info(self):
        basic_info_params = self.__generate_basic_info_params()
        return basic_info_params
    
    def advanced_info(self):
        pass
    
    def all_info(self):
        pass
    
    def __generate_basic_info_params(self):
        basic_info_params = dict(
            part='id',
            type=self.__type,
            q=self.__get_query(),
            maxResults=self.__MAX_RESULTS,
        ) 
        return basic_info_params
    
    def search_playlist(self, youtube_client, search_type='basic'):
        search_response = None
        if search_type == 'basic':
            basic_info_params = self.__generate_basic_info_params()
            search_request = youtube_client.search().list(
                **basic_info_params
            )
            search_response = search_request.execute()
            playlist_ids = self.__parse_playlists(search_response)
            play_lists = [FindPlaylist(playlist_id).find_playlist(youtube_client)
                         for playlist_id in playlist_ids]
        return play_lists
    
    def __parse_playlists(self, search_response):
        playlists_ids = []
        items = search_response['items']
        for item in items:
            playlists_ids.append(item['id']['playlistId'])
        return playlists_ids
        

In [22]:
class Channel:
    def __init__(self, id, title, description, customUrl, publishedAt, thumbnails, statistics):
        self.__id = id
        self.__title = title
        self.__description = description
        self.__customUrl = customUrl
        self.__publishedAt = publishedAt
        self.__thumbnails = thumbnails
        self.__statistics = statistics
        
    def get_channel_thumbnail(self):
        thumbnail = ''
        if self.__thumbnails:
            if self.__thumbnails.get('default'):
                thumbnail = self.__thumbnails.get('default').get('url')
            elif self.__thumbnails.get('medium'):
                thumbnail = self.__thumbnails.get('medium').get('url')
            elif self.__thumbnails.get('high'):
                thumbnail = self.__thumbnails.get('high').get('url')
            elif self.__thumbnails.get('standard'):
                thumbnail = self.__thumbnails.get('standard').get('url')
            elif self.__thumbnails.get('maxres'):
                thumbnail = self.__thumbnails.get('maxres').get('url')
        return thumbnail

In [97]:
class FindChannel:       
    def __init__(self, youtube_client):
        self.__youtube_client = youtube_client
        
    def __generate_basic_info_params(self, channel_id):
        basic_info_params = dict(
            id=channel_id,
            part='id,snippet,contentDetails,contentOwnerDetails,statistics,topicDetails',
        ) 
        return basic_info_params
    
    def find_channel_by_name(self):
        pass
    
    def find_channel_by_id(self, channel_id):
        """Find the video."""
        basic_info_params = self.__generate_basic_info_params(channel_id)
        search_request = self.__youtube_client.channels().list(
                **basic_info_params
            )
        search_response = search_request.execute()
        parsed_response = self.__parse_channel(search_response)
        youtube_channel = Channel(**parsed_response)
        return youtube_channel
    
    def __parse_channel(self, search_response):
        channel_details = {}
        items = search_response['items'][0]
        channel_details['id'] = items['id']
        channel_details['title'] = items['snippet']['title']
        channel_details['description'] = items['snippet']['description']
        channel_details['customUrl'] = items['snippet']['customUrl']
        channel_details['publishedAt'] = items['snippet']['publishedAt']
        channel_details['thumbnails'] = items['snippet']['thumbnails']
        channel_details['statistics'] = dict()
        channel_details['statistics']['viewCount'] = items['statistics']['viewCount']
        channel_details['statistics']['subscriberCount'] = items['statistics']['subscriberCount']
        channel_details['statistics']['videoCount'] = items['statistics']['videoCount']
        return channel_details

In [72]:
class YouTube:
    MAX_RESULTS = 20
    REGION_CODE = 'us'
    
    def __init__(self):
        self.__youtube_api_auth = YouTubeAPIAuth()
        self.__youtube_client = None
        self.__video_categories = []
        
    def get_credentials_path(self):
        return self.__youtube_api_auth.get_credentials_path()
        
    def authenticate_from_client_secrets_file(self, client_secrets_file: str, 
                credentials_path: str = ''):
        self.__youtube_client = self.__youtube_api_auth.authenticate_from_client_secrets_file(client_secrets_file, credentials_path)
        return self.__youtube_client
    
    def authenticate_from_credentials(self, credentials_path: str):
        self.__youtube_client = self.__youtube_api_auth.authenticate_from_credentials(credentials_path)
        return self.__youtube_client
    
    def generate_credentials(self, client_secrets_file: str, credentials_path: str = ''):
        self.__youtube_api_auth.generate_credentials(client_secrets_file, credentials_path)
        
    def get_youtube(self):
        return self.__youtube_client
    
    def get_video_categories(self):
        if not self.__video_categories:
            self.__video_categories = SearchYouTubeVideoCategories(self.__youtube_client).get_youtube_video_categories()
        return self.__video_categories
    
    def search_videos(self, query_string: str) -> list[str]:
        self.__video_search = VideoSearch(self.__youtube_client)
        self.__previous_page_token, self.__next_page_token, videos = self.__video_search.search_videos(query_string)
        return videos
    
    def get_video_collections(self):
        return self.__video_search.get_videos()
    
    def find_related_videos(self, youtube_video):
        related_videos = VideoSearch().search_related_videos(youtube_video, self.__youtube_client)
        return related_videos
    
    def find_most_popular_videos_by_region(self, region_code):
        most_popular_videos_by_region = VideoSearch().search_most_popular_videos_by_region(
            region_code, self.__youtube_client)
        return most_popular_videos_by_region
    
    def find_most_popular_videos_by_category(self, category_id):
        most_popular_videos_by_category = VideoSearch().search_most_popular_videos_by_category(
            category_id, self.__youtube_client)
        return most_popular_videos_by_category
    
    def find_video_by_url(self, video_url: str):
        """Get a specific video given the video url."""
        video_id = self.__get_video_id(video_url)
        return self.find_video_by_id(video_id)
        
    def find_video_by_id(self, video_id: str):
        """Find a video by id."""
        youtube_video = FindVideo(video_id).find_video(self.__youtube_client)
        return youtube_video
    
    @staticmethod
    def __get_video_id(video_url: str) -> str:
        """Get vdeo ID from video url"""
        if not video_url:
            raise ValueError('The video_ur has to be provided.')
        if not isinstance(video_url, str):
            raise TypeError('Te video_url has to be a string.')
        if '=' not in video_url:
            url_format = 'https://www.youtube.com/watch?v=Dqdu-FsBk0s'
            raise ValueError('Te video_url should be of the format "{url_format}"')
        video_url = video_url.split('=')[1]
        return video_url

In [73]:
youtube = YouTube()
client_secrets_file = '/home/lyle/Downloads/python_learning_site.json'
youtube_client = youtube.authenticate_from_client_secrets_file(client_secrets_file)

In [74]:
videos = youtube.search_videos('intermediate python programming')

In [99]:
find_video = FindVideo(youtube_client)
video = find_video.find_video('HGOBQPFzWKo')

AttributeError: 'FindChannel' object has no attribute 'find_channel'

In [96]:
video.get_video_comments()

[YouTubeComment(id='UgxhsgT0TB1L46coSCl4AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgzWip4i-I18FHGnrF14AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgzlYA9XcmFy3H4zvC14AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgwfYg96MRWRasIgZ994AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgzitDv1zd0IQ99hV2F4AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='Ugx8BIsFuOhfPCazerh4AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgzHNSC1BbAZxp-repR4AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgwMoSVbkHwom2S6g2R4AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgzJtNQPGQMbjAvINUB4AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),
 YouTubeComment(id='UgxbxWOQHIOCFM8Jfdh4AaABAg', videoId='HGOBQPFzWKo',         totalReplyCount=0),


In [76]:
video_collections = youtube.get_video_collections()

In [77]:
video_collections

In [78]:
video_collections.get_youtube_videos()

[YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={}),
 YouTubeVideo(video_details={})]

In [55]:
video = video_collections.get_youtube_videos()

In [85]:
video.to_dict()

{'video_id': 'HGOBQPFzWKo',
 'video_title': 'HGOBQPFzWKo',
 'video_description': 'Take your Python skills to the next level with this intermediate Python course. First, you will get a review of basic concepts such as lists, strings, and dictionaries, but with an emphasis on some lesser known capabilities. Then, you will learn more advanced topics such as threading, multiprocessing, context managers, generators, and more.\n\n💻 Code: https://github.com/python-engineer/python-engineer-notebooks/tree/master/advanced-python\n\n🎥 Course from Patrick Loeber. Check out his channel: https://www.youtube.com/channel/UCbXgNpp0jedKWcQiULLbDTA\n\n🔗 Written Tutorials from Patrick: https://www.python-engineer.com/courses/advancedpython/\n\n⭐️ Course Contents ⭐️\n⌨️ (0:00:00) Intro\n⌨️ (0:00:56) Lists\n⌨️ (0:16:30) Tuples\n⌨️ (0:29:49) Dictionaries\n⌨️ (0:42:40) Sets\n⌨️ (0:58:44) Strings\n⌨️ (1:22:50) Collections\n⌨️ (1:36:43) Itertools\n⌨️ (1:51:50) Lambda Functions\n⌨️ (2:04:03) Exceptions and Error